In [5]:
import numpy as np
import dash
from dash import dcc, html, Input, Output, State
import plotly.graph_objects as go

# Constants
map_size = 100
beacon_coord = (80, 80)
num_usvs = 5

# Initial USV coordinates (random)
initial_coords = [(np.random.randint(0, map_size), np.random.randint(0, map_size)) for _ in range(num_usvs)]

# Dash App
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Maritime Rescue Simulation"),
    
    # Input Section
    html.Div([
        html.Label("Select Coordinate Input Method:"),
        dcc.RadioItems(
            id='coordinate-method',
            options=[
                {'label': 'Custom Coordinates', 'value': 'custom'},
                {'label': 'Random Coordinates', 'value': 'random'},
                {'label': 'Live Custom Board (Simulated)', 'value': 'live'}
            ],
            value='random'
        ),
        
        html.Div(id='custom-coord-input', style={'display': 'none'}),
        
        html.Label("Select Geolocation Method:"),
        dcc.Dropdown(
            id='geolocation-method',
            options=[
                {'label': 'Power of Arrival', 'value': 'PoA'},
                {'label': 'Time of Arrival (ToA)', 'value': 'ToA'},
                {'label': 'Time Difference of Arrival (TDoA)', 'value': 'TDoA'},
                {'label': 'Angle of Arrival', 'value': 'AoA'},
                {'label': 'Phased Array', 'value': 'Phased'},
                {'label': 'Free Space Path Model', 'value': 'FS'}
            ],
            value='PoA'
        ),
        
        html.Button('Start Simulation', id='start-button', n_clicks=0)
    ], style={'width': '30%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding': '20px'}),
    
    # Visualization Section
    html.Div([
        dcc.Graph(id='simulation-graph')
    ], style={'width': '65%', 'display': 'inline-block'})
])

# Callback to show/hide custom coordinate input fields
@app.callback(
    Output('custom-coord-input', 'style'),
    [Input('coordinate-method', 'value')]
)
def toggle_custom_coord_input(method):
    if method == 'custom':
        return {'display': 'block'}
    return {'display': 'none'}

# Main callback for running the simulation
@app.callback(
    Output('simulation-graph', 'figure'),
    [Input('start-button', 'n_clicks')],
    [State('coordinate-method', 'value'), State('geolocation-method', 'value')]
)
def run_simulation(n_clicks, coord_method, geolocation_method):
    if n_clicks == 0:
        return go.Figure()

    # Determine starting coordinates based on user input
    if coord_method == 'custom':
        starting_coords = initial_coords  # Replace with user input handling
    elif coord_method == 'random':
        starting_coords = [(np.random.randint(0, map_size), np.random.randint(0, map_size)) for _ in range(num_usvs)]
    else:
        starting_coords = [(np.random.randint(0, map_size), np.random.randint(0, map_size)) for _ in range(num_usvs)]
    
    usv_coords = starting_coords
    fig = go.Figure()

    # Simulate movement
    for _ in range(20):
        beacon_estimate = np.mean(usv_coords, axis=0)
        usv_coords = [(x + np.sign(beacon_estimate[0] - x), y + np.sign(beacon_estimate[1] - y)) for x, y in usv_coords]
        
        usv_x, usv_y = zip(*usv_coords)
        fig.add_trace(go.Scatter(x=usv_x, y=usv_y, mode='markers', marker=dict(color='blue'), name='USVs'))
    
    # Add actual and estimated beacon location
    fig.add_trace(go.Scatter(x=[beacon_coord[0]], y=[beacon_coord[1]], mode='markers', marker=dict(color='red', size=12), name='Actual SOS Beacon'))
    fig.add_trace(go.Scatter(x=[beacon_estimate[0]], y=[beacon_estimate[1]], mode='markers', marker=dict(color='green', size=12), name='Estimated Beacon Location'))

    fig.update_layout(
        title=f"Maritime Rescue Simulation - Method: {geolocation_method}",
        xaxis_title="X Coordinate",
        yaxis_title="Y Coordinate",
        xaxis=dict(range=[0, map_size]),
        yaxis=dict(range=[0, map_size]),
    )

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)


In [6]:
import numpy as np
import dash
from dash import dcc, html, Input, Output, State
import plotly.graph_objects as go
import plotly.express as px

# Constants
map_size = 100
beacon_coord = (80, 80)
num_usvs = 5

# Initial USV coordinates (random)
initial_coords = [(np.random.randint(0, map_size), np.random.randint(0, map_size)) for _ in range(num_usvs)]

# Dash App
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    html.H1("Maritime Rescue Simulation Dashboard"),
    dcc.Tabs([
        dcc.Tab(label='Overview', children=[
            dcc.Graph(id='simulation-overview')
        ]),
        dcc.Tab(label='USV Path Analysis', children=[
            dcc.Dropdown(
                id='usv-selector',
                options=[{'label': f'USV {i+1}', 'value': i} for i in range(num_usvs)],
                value=list(range(num_usvs)),
                multi=True
            ),
            dcc.Graph(id='usv-path-trends')
        ]),
        dcc.Tab(label='Simulation Controls', children=[
            html.Label("Select Coordinate Input Method:"),
            dcc.RadioItems(
                id='coordinate-method',
                options=[
                    {'label': 'Custom Coordinates', 'value': 'custom'},
                    {'label': 'Random Coordinates', 'value': 'random'},
                    {'label': 'Live Custom Board (Simulated)', 'value': 'live'}
                ],
                value='random'
            ),
            html.Div(id='custom-coord-input', style={'display': 'none'}),
            
            html.Label("Select Geolocation Method:"),
            dcc.Dropdown(
                id='geolocation-method',
                options=[
                    {'label': 'Power of Arrival', 'value': 'PoA'},
                    {'label': 'Time of Arrival (ToA)', 'value': 'ToA'},
                    {'label': 'Time Difference of Arrival (TDoA)', 'value': 'TDoA'},
                    {'label': 'Angle of Arrival', 'value': 'AoA'},
                    {'label': 'Phased Array', 'value': 'Phased'},
                    {'label': 'Free Space Path Model', 'value': 'FS'}
                ],
                value='PoA'
            ),
            
            html.Button('Start Simulation', id='start-button', n_clicks=0),
            dcc.Graph(id='simulation-graph')
        ])
    ])
])

# Callback to show/hide custom coordinate input fields
@app.callback(
    Output('custom-coord-input', 'style'),
    [Input('coordinate-method', 'value')]
)
def toggle_custom_coord_input(method):
    if method == 'custom':
        return {'display': 'block'}
    return {'display': 'none'}

# Main callback for running the simulation
@app.callback(
    Output('simulation-graph', 'figure'),
    [Input('start-button', 'n_clicks')],
    [State('coordinate-method', 'value'), State('geolocation-method', 'value')]
)
def run_simulation(n_clicks, coord_method, geolocation_method):
    if n_clicks == 0:
        return go.Figure()

    # Determine starting coordinates based on user input
    if coord_method == 'custom':
        starting_coords = initial_coords  # Replace with user input handling
    elif coord_method == 'random':
        starting_coords = [(np.random.randint(0, map_size), np.random.randint(0, map_size)) for _ in range(num_usvs)]
    else:
        starting_coords = [(np.random.randint(0, map_size), np.random.randint(0, map_size)) for _ in range(num_usvs)]
    
    usv_coords = starting_coords
    fig = go.Figure()

    # Simulate movement
    for _ in range(20):
        beacon_estimate = np.mean(usv_coords, axis=0)
        usv_coords = [(x + np.sign(beacon_estimate[0] - x), y + np.sign(beacon_estimate[1] - y)) for x, y in usv_coords]
        
        usv_x, usv_y = zip(*usv_coords)
        fig.add_trace(go.Scatter(x=usv_x, y=usv_y, mode='markers', marker=dict(color='blue'), name='USVs'))
    
    # Add actual and estimated beacon location
    fig.add_trace(go.Scatter(x=[beacon_coord[0]], y=[beacon_coord[1]], mode='markers', marker=dict(color='red', size=12), name='Actual SOS Beacon'))
    fig.add_trace(go.Scatter(x=[beacon_estimate[0]], y=[beacon_estimate[1]], mode='markers', marker=dict(color='green', size=12), name='Estimated Beacon Location'))

    fig.update_layout(
        title=f"Maritime Rescue Simulation - Method: {geolocation_method}",
        xaxis_title="X Coordinate",
        yaxis_title="Y Coordinate",
        xaxis=dict(range=[0, map_size]),
        yaxis=dict(range=[0, map_size]),
    )

    return fig

# Callback to update the overview of the simulation
@app.callback(
    Output('simulation-overview', 'figure'),
    Input('simulation-graph', 'figure')
)
def update_simulation_overview(simulation_figure):
    return simulation_figure

# Callback to update USV path trends
@app.callback(
    Output('usv-path-trends', 'figure'),
    [Input('usv-selector', 'value')],
    [State('simulation-graph', 'figure')]
)
def update_usv_path_trends(selected_usvs, simulation_figure):
    if not simulation_figure or not selected_usvs:
        return go.Figure()
    
    fig = go.Figure()
    for i in selected_usvs:
        usv_trace = simulation_figure['data'][i]
        fig.add_trace(go.Scatter(x=usv_trace['x'], y=usv_trace['y'], mode='lines+markers', name=f'USV {i+1}'))
    
    fig.update_layout(
        title='USV Path Trends',
        xaxis_title='X Coordinate',
        yaxis_title='Y Coordinate',
        xaxis=dict(range=[0, map_size]),
        yaxis=dict(range=[0, map_size]),
    )

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
